<a href="https://colab.research.google.com/github/atveit/AIDemoUI/blob/main/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-2.0-flash-exp' # @param {isTemplate: true}
contents_b64 = 'W10=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[]


## Call `generate_content`

In [4]:
from IPython.display import display
from IPython.display import Markdown

contents = "Generate a story about a poodle in 2 scenes and one image per scene"

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

Okay, here's a story about a poodle in two scenes, with an image description for each:

**Scene 1: The Agony of the Awkward Trim**

**Story:**

Percy, a standard poodle of the fluffiest, most cloud-like variety, was having a crisis. Usually, he pranced with the confidence of a canine king, his white curls bouncing with every majestic step. Today, however, he was a study in misery. He'd just returned from his regular grooming appointment, and something had gone terribly, terribly wrong. Instead of the elegant, poofy coif he was accustomed to, he was sporting a…thing. A strange, sculpted, almost geometric…thing. His once-noble head looked like a topiary gone rogue. Worst of all, the groomer, bless her heart, had decided that he needed "a bit more personality" and had added little pom-poms to his hindquarters. Percy stood rigidly by the living room window, his tail tucked tightly between his legs, contemplating the sheer indignity of it all. He could see squirrels chattering in the backyard, their little tails twitching in what he could only assume was amusement at his expense. The image of him reflected in the window was just too much. He let out a low, mournful groan. The world was cruel, and he was, undeniably, a poodle in distress.

**Image Description:**

**Image 1:** A full shot of a large, white standard poodle named Percy, standing rigidly in front of a large bay window.  He is positioned slightly to the left of the frame, facing toward the window but with his head angled down in an expression of defeat. Sunlight streams in through the window, highlighting the awkward, overly sculpted haircut on his head.  His fur looks choppy and uneven, and two comical, rounded pom-poms are attached to his rear.  He has a downturned mouth and is casting a forlorn gaze toward the floor.  Outside the window, we can see the blurred outlines of trees, a grassy backyard, and a few squirrels seemingly chattering on a branch. The room is comfortably furnished, with a plush rug and a glimpse of a couch behind Percy, suggesting a well-loved home. The overall feeling is one of humor tinged with pathos.

**Scene 2: The Redemption of the Runway**

**Story:**

Three weeks had passed. Three weeks of agonizing over his coiffure, three weeks of hiding his pom-poms, three weeks of refusing all forms of affection (except the occasional secret ear scratch from his human). Then, an invitation arrived. Not just any invitation, but one to the annual "Paws & Claws Fashion Show" at the local park. Percy's human, ever the optimist, thought it would be good for his confidence. Percy was dubious. He'd imagined strutting his stuff with grace and sophistication but felt doomed to be the laughingstock in his current state. But, as he was being led down the makeshift runway, something shifted. The crowd cheered. Not a pity cheer. A genuine cheer. And then, it clicked. It wasn’t about the cut. It was about *him*. He was a poodle. A magnificent, albeit slightly geometrically-challenged poodle. He straightened his back, raised his chin, and with a flick of his still-rather-ridiculous pom-pom tail, Percy transformed into a runway star. He pranced, he posed, he even managed a perfectly executed 'sit-pretty' maneuver that sent the crowd wild. The awkward trim became his signature. The pom-poms, his trademark. Percy, once ashamed, had finally found his groove. 

**Image Description:**

**Image 2:**  A slightly wider shot than the previous, this image captures Percy mid-strut on a makeshift runway set up in a sunny park. A simple red carpet leads towards the camera. Percy, now beaming with confidence, is mid-stride, his head held high and his tail (with its pom-poms!) held jauntily. He is surrounded by a crowd of people, some of whom are taking photos and smiling broadly.  The scene is vibrant and full of life, with lush green trees forming a backdrop. Other well-groomed dogs can be seen waiting their turn, but they are blurred in the background, keeping the focus solely on Percy.  The lighting is bright, and the feeling is one of joyful triumph.

Hopefully, this captures the story you were looking for! Let me know if you have any other requests.


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

# GEMINI WITH MULTIPLE IMAGES INLINED IN TEXT

In [6]:
import base64
import json

# Example for generating text with multiple images

contents = [
        {
            "role": "user",  # The role is "user"
            "parts": [
                {
                    "text": "Generate a story about a poodle in 2 scenes and one image per scene"
                },
            ]
        },
]

# Convert the contents to base64
contents_b64 = base64.b64encode(json.dumps(contents).encode()).decode()

# Call the generate_content method
response = gemini.generate_content(
    contents_b64,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

In [7]:
def show_file(file_data):
        mime_type = file_data["mime_type"]

        if data := file_data.get("inline_data", None):
            if mime_type.startswith("image/"):
                image = IPython.display.Image(data=data, width=256)
                IPython.display.display(image)
                print()
                return
            else:
                print(f"File with mime_type: {mime_type} not supported")
                return

        print(f"File data not found or not supported")

In [10]:
print(response.candidates)

[content {
  parts {
    text: "Okay, here\'s a story about a poodle in two scenes, with a description of an image for each:\n\n**Scene 1:**\n\nThe late afternoon sun cast long, lazy shadows across Mrs. Higgins\' manicured lawn.  Fluffy, a purebred apricot poodle with a perpetually curious tilt to his head, was engaged in his favorite pastime: observing the world from the sun-drenched patio.  He wasn\'t interested in squirrels or butterflies today; he was focused on the little red ball that had rolled under the rose bushes.  His nose twitched, his tail gave a tentative wag, and his long, fluffy ears drooped slightly as he tried to gauge the distance. He nudged a nearby terracotta pot with his nose, then whined softly, frustrated by the prickly barrier of thorns.  Fluffy was a master of patience, however, and he knew that with enough strategic maneuvering, that ball would be his.\n\n**Image for Scene 1:**\n\n*   **Description:** A wide shot of a manicured lawn in golden afternoon light.